# Classifier Model Exercise 

In [46]:
import pandas as pd

In [47]:
census_data = pd.read_csv('../DataSet/adult.csv')

In [48]:
census_data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [49]:
census_data['income'].unique()

array(['<=50K', '>50K'], dtype=object)

In [50]:
def label_fix(label):
    if label == '<=50K':
        return 0
    else:
        return 1

In [51]:
census_data['income'] = census_data['income'].apply(label_fix)

In [52]:
from sklearn.model_selection import train_test_split

In [53]:
y_data = census_data['income']

In [54]:
y_data.head()

0    0
1    0
2    0
3    0
4    0
Name: income, dtype: int64

In [55]:
census_data.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'income'],
      dtype='object')

In [56]:
x_data = census_data.drop(['fnlwgt','income', 'race'], axis=1)

In [57]:
x_data.head()

,age,workclass,education,education.num,marital.status,occupation,relationship,sex,capital.gain,capital.loss,hours.per.week,native.country
0,90,?,HS-grad,9,Widowed,?,Not-in-family,Female,0,4356,40,United-States
1,82,Private,HS-grad,9,Widowed,Exec-managerial,Not-in-family,Female,0,4356,18,United-States
2,66,?,Some-college,10,Widowed,?,Unmarried,Female,0,4356,40,United-States
3,54,Private,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,Female,0,3900,40,United-States
4,41,Private,Some-college,10,Separated,Prof-specialty,Own-child,Female,0,3900,40,United-States


In [58]:
x_data.columns

Index(['age', 'workclass', 'education', 'education.num', 'marital.status',
       'occupation', 'relationship', 'sex', 'capital.gain', 'capital.loss',
       'hours.per.week', 'native.country'],
      dtype='object')

In [59]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=.3)

In [60]:
import tensorflow as tf

In [61]:
gender = tf.feature_column.categorical_column_with_vocabulary_list('sex', ['Female', 'Male'])
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation', hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket('marital.status', hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket('relationship', hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket('education', hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket('workclass', hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket('native.country', hash_bucket_size=1000)
age = tf.feature_column.numeric_column('age')
education_num = tf.feature_column.numeric_column('education.num')
capital_gain = tf.feature_column.numeric_column('capital.gain')
capital_loss = tf.feature_column.numeric_column('capital.loss')
hours_per_week = tf.feature_column.numeric_column('hours.per.week')

In [62]:
feat_cols = [gender,occupation,marital_status,relationship,education,
             workclass,native_country,age,education_num,capital_gain,
             capital_gain, hours_per_week] 

In [63]:
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train, y=y_train, 
                                                 batch_size=100, num_epochs=None, 
                                                 shuffle=True)

In [64]:
eval_func = tf.estimator.inputs.pandas_input_fn(x=x_test, y=y_test,
                                                batch_size=100, num_epochs=None,
                                               shuffle=False)

In [65]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/kp/vv_v1p3j095gw56rm7c41_sh0000gn/T/tmpkawtm0eo', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13cc72fd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [66]:
model.train(input_fn=input_func, steps=100)

INFO:tensorflow:Calling model_fn.


ValueError: Duplicate feature column name found for columns: NumericColumn(key='capital.gain', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None) and NumericColumn(key='capital.gain', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None). This usually means that these columns refer to same base feature. Either one must be discarded or a duplicated but renamed item must be inserted in features dict.